In [1]:
import sys

from torch import nn
from torch.utils.data import DataLoader
from torchvision.models import mobilenet_v3_large
from torchvision.transforms import Resize
import torch

sys.path.insert(0, '..')

## View MobileNetV3 Architecture for Reference

In [2]:
# weights = MobileNet_V3_Large_Weights.IMAGENET1K_V2
model = mobilenet_v3_large(pretrained=True)

print(model)

MobileNetV3(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False

## Remove Last Layer

In [3]:
model_submodule_excl_last = list(model.children())[:-1]
model_submodule_excl_last = nn.Sequential(*model_submodule_excl_last)

model_submodule_excl_last

Sequential(
  (0): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      

In [4]:
model_submodule_excl_last(torch.rand(10, 3, 224, 224)).shape

torch.Size([10, 960, 1, 1])

In [5]:
model_last_submodule = list(list(model.children())[-1].children())[:-1]  # Remove last layer from last submodule
model_last_submodule = nn.Sequential(*model_last_submodule)

model_last_submodule

Sequential(
  (0): Linear(in_features=960, out_features=1280, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
)

In [6]:
# output = model_submodule_excl_last(torch.rand(10, 3, 224, 224))
# model_last_submodule(output)

In [7]:
output = model_submodule_excl_last(torch.rand(10, 3, 224, 224))
output = torch.flatten(output, 1)
model_last_submodule(output).shape

torch.Size([10, 1280])

## Import `SiameseDuplicateImageNetwork` for Testing

In [8]:
from scripts.dataset_duplicate_image import DuplicateImageDataset
from scripts.model import SiameseDuplicateImageNetwork

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [10]:
model = SiameseDuplicateImageNetwork().to(device)

In [11]:
print(model)

SiameseDuplicateImageNetwork(
  (model): Sequential(
    (0): SiameseNetwork(
      (net1): Sequential(
        (0): Sequential(
          (0): Sequential(
            (0): ConvNormActivation(
              (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
              (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
              (2): Hardswish()
            )
            (1): InvertedResidual(
              (block): Sequential(
                (0): ConvNormActivation(
                  (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
                  (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
                  (2): ReLU(inplace=True)
                )
                (1): ConvNormActivation(
                  (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
                  (1): BatchNorm2d(16, eps=0.001, mom

In [12]:
train_dataset = DuplicateImageDataset('../data/Training Data', transforms=[Resize((224, 224), antialias=True)])
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [13]:
feature1, feature2, _ = next(iter(train_dataloader))
feature1, feature2

(tensor([[[[0.6543, 0.6687, 0.6789,  ..., 0.7089, 0.6976, 0.6890],
           [0.6628, 0.6721, 0.6835,  ..., 0.6955, 0.6837, 0.6745],
           [0.6671, 0.6769, 0.6845,  ..., 0.6766, 0.6699, 0.6644],
           ...,
           [0.3491, 0.3544, 0.3669,  ..., 0.2606, 0.2547, 0.2428],
           [0.3450, 0.3512, 0.3627,  ..., 0.2533, 0.2521, 0.2325],
           [0.3366, 0.3482, 0.3572,  ..., 0.2634, 0.2576, 0.2311]],
 
          [[0.6543, 0.6687, 0.6789,  ..., 0.7214, 0.7094, 0.7007],
           [0.6628, 0.6721, 0.6835,  ..., 0.7092, 0.6954, 0.6863],
           [0.6671, 0.6769, 0.6845,  ..., 0.6904, 0.6817, 0.6762],
           ...,
           [0.3216, 0.3270, 0.3356,  ..., 0.3213, 0.3175, 0.3056],
           [0.3175, 0.3237, 0.3314,  ..., 0.3141, 0.3149, 0.2952],
           [0.3091, 0.3208, 0.3260,  ..., 0.3242, 0.3204, 0.2938]],
 
          [[0.6465, 0.6609, 0.6711,  ..., 0.7395, 0.7290, 0.7203],
           [0.6550, 0.6642, 0.6756,  ..., 0.7234, 0.7150, 0.7059],
           [0.6593, 0.66

In [14]:
feature1 = feature1.to(device)
feature2 = feature2.to(device)

output = model(feature1, feature2)
print(output)

tensor([[-0.0483],
        [-0.0488],
        [-0.0531],
        [-0.0486],
        [-0.0515],
        [-0.0477],
        [-0.0533],
        [-0.0489],
        [-0.0509],
        [-0.0477],
        [-0.0532],
        [-0.0488],
        [-0.0512],
        [-0.0479],
        [-0.0504],
        [-0.0483],
        [-0.0508],
        [-0.0494],
        [-0.0484],
        [-0.0499],
        [-0.0488],
        [-0.0528],
        [-0.0502],
        [-0.0512],
        [-0.0497],
        [-0.0492],
        [-0.0490],
        [-0.0495],
        [-0.0519],
        [-0.0501],
        [-0.0507],
        [-0.0499],
        [-0.0488],
        [-0.0493],
        [-0.0508],
        [-0.0517],
        [-0.0490],
        [-0.0496],
        [-0.0473],
        [-0.0508],
        [-0.0497],
        [-0.0490],
        [-0.0514],
        [-0.0490],
        [-0.0493],
        [-0.0500],
        [-0.0478],
        [-0.0528],
        [-0.0513],
        [-0.0506],
        [-0.0477],
        [-0.0482],
        [-0.

In [15]:
output.shape

torch.Size([64, 1])